# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  4.94it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.01it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.47it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.31it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.49it/s]



...

 83%|████████▎ | 19/23 [00:06<00:00,  4.15it/s]

 87%|████████▋ | 20/23 [00:06<00:00,  3.78it/s]

 91%|█████████▏| 21/23 [00:06<00:00,  3.84it/s]

 96%|█████████▌| 22/23 [00:06<00:00,  3.83it/s]

100%|██████████| 23/23 [00:07<00:00,  3.20it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Emily and I am the owner and primary instructor of Bloom Yoga Studio. I am passionate about sharing the benefits of yoga with my community and helping each student develop a personal practice that nourishes their body, mind, and spirit.
I have been practicing yoga for over 10 years and have completed numerous training programs to become a certified yoga instructor. I hold a 500-hour certification in Hatha and Vinyasa Yoga through the Yoga Alliance. My teaching style is compassionate, clear, and adaptable to meet the needs of each student.
I believe that yoga is a journey, not a destination, and that every body is capable of growth and transformation
Prompt: The president of the United States is
Generated text:  the head of state and head of government of the United States. The president serves a four-year term and is elected by the people through the Electoral College. The president is responsible for appointing federal judges, ambassadors, an

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Wendy

,

 and

 I

...

 people

 around

 the

 world

.

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Jaycee and I am a proud and devoted daughter, sister, and friend. I am a strong believer in the importance of family and the value of every individual within it. I am passionate about making a difference in the lives of others, whether it be through volunteering, mentoring, or simply being a supportive and caring presence in someone's life. I am a bit of a hopeless romantic at heart and believe that love and kindness can conquer even the toughest of challenges.
I am a bit of a goofball and enjoy making people laugh. I am also a bit of a bookworm and love getting lost in a good novel or learning new things

Prompt: The capital of France is
Generated text: , of course, Paris. But have you ever wondered what makes this city so special? Here are some of the reasons why Paris is the City of Light.
One of the most famous landmarks in Paris is the Eiffel Tower. This iconic iron structure was built for the 1889 World’s Fair and stands 324 meters tall

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Diana

 and

 I

 am

...

 lead

 to

 AI

 systems

 that

In [6]:
llm.shutdown()